### Set partitioning problem

Involves partitioning a set of items into subsets so that
each item appears in exactly one subset and the cost of the subsets chosen is minimized

In [29]:
from qiskit_optimization import QuadraticProgram
from qiskit import BasicAer
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import QuantumInstance
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit_algorithms.optimizers import COBYLA

$ min (Y) = 3.x_1 + 2.x_2 + x_3 + x_4 + 3.x_5 + 2.x_6 $ 

#### Penalty 
P = -50

#### Constraints 
- $ x_1 + x_3 + x_6 = 1 $
- $ x_2 + x_3 + x_5 + x_6 = 1 $
- $ x_3 + x_4 + x_5 = 1 $
- $ x_1 + x_2 + x_4 + x_6 = 1 $

Considering constraints with a penalty, they would look like below. 
 
$ min (y) =  3.x_1 + 2.x_2 + x_3 + x_4 + 3.x_5 + 2.x_6 + P(x_1 + x_3 + x_6 - 1)^2 + P( x_2 + x_3 + x_5 + x_6 - 1)^2 + P(x_3 + x_4 + x_5 - 1)^2 + P(x_1 + x_2 + x_4 + x_6 -1)^2 $

Subsitute $ x_i = x_i^2 $

$ min (y) =  3.x_1^2 + 2.x_2^2 + x_3^2 + x_4^2 + 3.x_5^2 + 2.x_6^2 - 50(x_1 + x_3 + x_6 - 1)^2 - 50( x_2 + x_3 + x_5 + x_6 - 1)^2 - 50(x_3 + x_4 + x_5 - 1)^2 - 50(x_1 + x_2 + x_4 + x_6 -1)^2 $

$ min (y) =  -97 x_1^2 + 100 x_1 x_2 + 100 x_1 x_3 + 100 x_1 x_4 + 200 x_1 x_6 + -98 x_2^2 + 100 x_2 x_3 + 100 x_2 x_4 + 100 x_2 x_5 + 200 x_2 x_6 + -149 x_3^2 + 100 x_3 x_4 + 200 x_3 x_5 + 200 x_3 x_6 + -99 x_4^2 + 100 x_4 x_5 + 100 x_4 x_6 + -97 x_5^2 + 100 x_5 x_6 + -148 x_6^2 + 200 $

Drop the 200, use the rest for quadratic expression and use it later for substraction.

In [30]:
subset_cost = [3,2,1,1,3,2]
constraints = [
    [[1,3,6], [1]],
    [[2,3,5,6], [1]],
    [[3,4,5], [1]],
    [[1,2,4,6], [1]]
]
mod = QuadraticProgram("set_partitioning")
[mod.binary_var("x"+str(i+1)) for i in range(6)]
cnt = 1
for c,p in constraints:
    linear_constraints = {}
    for k in c:
        linear_constraints["x"+str(k)] = p[0]
    mod.linear_constraint(linear=linear_constraints, sense="==", rhs=1, name="lin_eq" + str(cnt))
    cnt += 1

mod.maximize(quadratic={
    ("x1", "x1"):-97, 
    ("x2", "x2"): -98, 
    ("x3", "x3"): -149, 
    ("x4", "x4"): -99, 
    ("x5", "x5"): -97, 
    ("x6", "x6"): -148, 
    ("x1", "x2"):100,
    ("x1", "x3"):100,("x2", "x3"):100,
    ("x1", "x4"):100,("x2", "x4"):100,("x3", "x4"):100,
    ("x1", "x6"):200,("x2", "x5"):100,("x2", "x6"):200,("x3", "x5"):200,("x3", "x6"):200,("x4", "x5"):100,("x4", "x6"):100,("x5", "x6"):100,
})

print(mod.prettyprint())


Problem name: set_partitioning

Maximize
  -97*x1^2 + 100*x1*x2 + 100*x1*x3 + 100*x1*x4 + 200*x1*x6 - 98*x2^2 + 100*x2*x3
  + 100*x2*x4 + 100*x2*x5 + 200*x2*x6 - 149*x3^2 + 100*x3*x4 + 200*x3*x5
  + 200*x3*x6 - 99*x4^2 + 100*x4*x5 + 100*x4*x6 - 97*x5^2 + 100*x5*x6 - 148*x6^2

Subject to
  Linear constraints (4)
    x1 + x3 + x6 == 1  'lin_eq1'
    x2 + x3 + x5 + x6 == 1  'lin_eq2'
    x3 + x4 + x5 == 1  'lin_eq3'
    x1 + x2 + x4 + x6 == 1  'lin_eq4'

  Binary variables (6)
    x1 x2 x3 x4 x5 x6



In [31]:
quantum_instance = Sampler()
qaoa_mes = QAOA(quantum_instance, COBYLA(), reps=2)
qaoa = MinimumEigenOptimizer(qaoa_mes)

qaoa_result = qaoa.solve(mod)
[qaoa_result.x], [200+qaoa_result.fval]


([array([1., 0., 0., 0., 1., 0.])], [6.0])